In [ ]:
import numpy as np 
import pandas as pd 

import os
import glob
import skimage.io

import skimage
import tifffile
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import multiprocessing

multiprocessing.cpu_count()

In [ ]:
BASE_FOLDER = "/kaggle/input/prostate-cancer-grade-assessment/"
!ls {BASE_FOLDER}

In [ ]:
train_df = pd.read_csv(BASE_FOLDER + "train.csv")
train_df.head()

In [ ]:
TRAIN = "train_images/"
TRAIN_MASKS = "train_label_masks/"

images = glob.glob(BASE_FOLDER+"train_images/*")
masks = glob.glob(BASE_FOLDER+"train_label_masks/*")
images = set(images)
masks = set(masks)
print(len(images))
print(len(masks))
print(len(images))


In [ ]:
# Leer una imagen
image = tifffile.imread(BASE_FOLDER + "train_images/" + train_df["image_id"][0] + ".tiff", key=1)
mask = tifffile.imread(BASE_FOLDER + "train_label_masks/" + train_df["image_id"][0] + "_mask.tiff", key=1)

plt.imshow(image)
print(image.shape)

In [ ]:
# Aunque también se guarda como una imagen de tres colores,
# la máscara solo tiene información en el primero
print (mask.shape)
for i in range(3):
    plt.subplot(1,3,i+1)
    plt.imshow(mask[:,:,i])

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(image)
plt.imshow(mask[:, :, 0], alpha=0.4)

In [ ]:
# Convertir a grises
im_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
plt.imshow(im_gray, cmap="gray")

In [ ]:
# Binarizar
(T, im_bw) = cv2.threshold(im_gray, 200, 255, cv2.THRESH_BINARY_INV)
shp = im_bw.shape
im_bw[:122,:] = 0
im_bw[shp[0]-122:shp[0],:] = 0
im_bw[:,:122] = 0
im_bw[:,shp[1]-122:shp[1]] = 0
plt.imshow(im_bw, cmap="gray")

In [ ]:
# Encontrar localización de los pixeles y seleccionar
# aleatoriamente algunos de ellos
idx = np.nonzero(im_bw.ravel())[0]
idx = np.column_stack(np.unravel_index(idx, im_bw.shape))
candidates = idx[np.random.randint(0, len(idx), 49), :]

In [ ]:
# Mostrar imagen y los puntos seleccionados
plt.figure(figsize=(10,10))
plt.imshow(im_bw, cmap="gray")
plt.scatter(candidates[:, 1], candidates[:, 0], s=10, c="r")

In [ ]:
# Función cuya única entrada es la imagen
SZ_IM = 112
TILES = 49
def tiles_rand(file, folder="train_images/", mask=False):
    
    
    image_mid = tifffile.imread(BASE_FOLDER +
                            folder +
                            file + ".tiff",
                            key=1)
    
    # Pasar a blanco y negro
    im_gray = cv2.cvtColor(image_mid, cv2.COLOR_BGR2GRAY)
    
    # Thresholding para binarizar
    (T, threshInv) = cv2.threshold(im_gray, 200, 255, cv2.THRESH_BINARY_INV)
    
    # Evitar seleccionar los pixeles de los bordes
    shp = threshInv.shape
    threshInv[:112,:] = 0
    threshInv[shp[0]-112:shp[0],:] = 0
    threshInv[:,:112] = 0
    threshInv[:,shp[1]-112:shp[1]] = 0
    
    # Encontrar pixeles mascara
    idx = np.nonzero(threshInv.ravel())[0]
    idx = np.column_stack(np.unravel_index(idx, threshInv.shape))
    candidates = idx[np.random.randint(0, len(idx), TILES), :]
    
    # Seccionar la imagen
    tiles = []
    sz_md = int(SZ_IM/2)
    for y, x in candidates:
        tiles.append(image_mid[y-sz_md:y+sz_md, x-sz_md:x+sz_md])
        
    if mask:
        masks = []
        mask = tifffile.imread(BASE_FOLDER +
                               TRAIN_MASKS +
                               file + "_mask.tiff",
                               key=1)
        for y, x in candidates:
            masks.append(mask[y-sz_md:y+sz_md, x-sz_md:x+sz_md, 0])
        
        return np.array(tiles), np.array(masks)
    else:
        return np.array(tiles)

tiles, masks = tiles_rand(train_df["image_id"][0], mask=True)
np.shape(tiles)

In [ ]:
# Función cuya única entrada es la imagen
SZ_IM = 112
TILES = 49
kernel = np.array([[0,1,0], [1,1,1], [0,1,0]], np.uint8)#np.ones((1, 1), np.uint8)
def tiles_rand2(file, folder="train_images/", mask=False, versions=None):
    
    image_mid = tifffile.imread(BASE_FOLDER +
                            folder +
                            file + ".tiff",
                            key=1)
    #print(image_mid.shape)
    
    scale_percent = 5 # percent of original size
    width = int(image_mid.shape[1] * scale_percent / 100)
    height = int(image_mid.shape[0] * scale_percent / 100)
    dim = (width, height)
    
    image_min = cv2.resize(image_mid, dim, cv2.INTER_NEAREST)
    
    # Pasar a blanco y negro
    im_gray = cv2.cvtColor(image_min, cv2.COLOR_BGR2GRAY)
    
    # Thresholding para binarizar
    (T, threshInv) = cv2.threshold(im_gray, 200, 255, cv2.THRESH_BINARY_INV)
    threshInv = cv2.morphologyEx(threshInv, cv2.MORPH_CLOSE, kernel)
    
    #plt.imshow(threshInv)
    
    
    # Evitar seleccionar los pixeles de los bordes
    shp = threshInv.shape
    pad = int(122 * scale_percent / 100)
    threshInv[:pad,:] = 0
    threshInv[shp[0]-pad:shp[0],:] = 0
    threshInv[:,:pad] = 0
    threshInv[:,shp[1]-pad:shp[1]] = 0
    
    # Encontrar pixeles mascara
    idx = np.nonzero(threshInv.ravel())[0]
    idx = np.column_stack(np.unravel_index(idx, threshInv.shape))
    candidates = idx[np.random.randint(0, len(idx), TILES), :] * 20
    
    #plt.imshow(threshInv, cmap="gray")
    #plt.scatter(candidates[:, 1]/10, candidates[:, 0]/10, s=10, c="r")
    
    # Seccionar la imagen
    tiles = []
    sz_md = int(SZ_IM/2)
    for y, x in candidates:
        tiles.append(image_mid[y-sz_md:y+sz_md, x-sz_md:x+sz_md])
        
    if mask:
        masks = []
        mask = tifffile.imread(BASE_FOLDER +
                               TRAIN_MASKS +
                               file + "_mask.tiff",
                               key=1)
        for y, x in candidates:
            masks.append(mask[y-sz_md:y+sz_md, x-sz_md:x+sz_md, 0])
        
        return np.array(tiles), np.array(masks), candidates
    else:
        return np.array(tiles)

tiles, masks, candidates = tiles_rand2(train_df["image_id"][2], mask=True)
np.shape(tiles)

In [ ]:
def to_mosaic(tiles):
    dims = int(np.sqrt(TILES))
    tiles = np.array(tiles)
    mosaic = tiles.reshape(dims, dims, SZ_IM, SZ_IM, 3)
    mosaic = mosaic.swapaxes(1, 2)
    mosaic = mosaic.reshape(SZ_IM * dims, SZ_IM * dims, 3)
    
    return mosaic
    
mosaic = to_mosaic(tiles)
np.shape(mosaic)
plt.imshow(mosaic)

In [ ]:
#print(idx)
plt.figure(figsize=(5,5))
im = tifffile.imread(BASE_FOLDER + "train_images/" + train_df["image_id"][2] + ".tiff", key=1)
#print(im.shape)
print(im.dtype)
plt.imshow(im)
plt.scatter(candidates[:, 1], candidates[:, 0], s=10, c="r")

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(49):
    plt.subplot(7,7,i+1)
    plt.imshow(tiles[i])
    plt.axis("off")

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(49):
    plt.subplot(7,7,i+1)
    plt.imshow(masks[i], vmin=0)
    plt.axis("off")

In [ ]:
plt.figure(figsize=(12, 12))
for i in range(49):
    plt.subplot(7,7,i+1)
    plt.imshow(tiles[i])
    plt.imshow(masks[i], alpha=0.4, vmin=0)
    plt.axis("off")

In [ ]:
idd = 10503
print(train_df["image_id"][idd])

im = tifffile.imread(BASE_FOLDER + "train_images/" + train_df["image_id"][idd] + ".tiff", key=1)
print(im.shape)
print(train_df["gleason_score"][idd])

In [ ]:
import timeit
for i in range(5):
    
    #time1 = timeit.timeit('tiles_rand("0005f7aaab2800f6170c399693a96917", mask=True)', "from __main__ import tiles_rand", number=500)
    time2 = timeit.timeit('tiles_rand2("fd6fe1a3985b17d067f2cb4d5bc1e6e1", mask=True)', "from __main__ import tiles_rand2", number=1)
    print(time2)

In [ ]:
#print(time1)
print(time2/5)

In [ ]:
from PIL import Image
!mkdir mosaics

In [ ]:
categories = train_df.gleason_score.unique()
for categorie in categories:
    names = train_df[train_df["gleason_score"]==categorie].head()["image_id"]
    for name in names:
        tiles = tiles_rand2(name)
        mosaic = to_mosaic(tiles)
        im = Image.fromarray(mosaic)
        im.save("mosaics/"+name+".png")
        

In [ ]:
train_df.gleason_score.unique()